<a href="https://colab.research.google.com/github/galihweb000haha/CNN/blob/main/groceries_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision import transforms, datasets

In [ ]:
!git clone https://github.com/rianrajagede/groceries_classification.git

Cloning into 'groceries_classification'...
remote: Enumerating objects: 1142, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1142 (delta 6), reused 16 (delta 3), pack-reused 1121
Receiving objects: 100% (1142/1142), 199.16 MiB | 41.49 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
! wget -c https://github.com/rianrajagede/groceries_classification -O groceries_classification.zip
! unzip groceries_classification.zip
! rm groceries_classification.zip

--2022-03-22 19:18:35--  https://github.com/rianrajagede/groceries_classification
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘groceries_classification.zip’

groceries_classific     [ <=>                ] 154.21K  --.-KB/s    in 0.05s   

2022-03-22 19:18:36 (3.07 MB/s) - ‘groceries_classification.zip’ saved [157907]

Archive:  groceries_classification.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of groceries_classification.zip or
        groceries_classification.zip.zip, and cannot find groceries_classification.zip.ZIP, period.


In [ ]:
from torchvision.transforms.transforms import ToTensor
train_dir = "groceries_classification/train/"
test_dir = "groceries_classification/test"

train_transform = transforms.Compose([
    transforms.CenterCrop(150),
    transforms.RandomRotation(5),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = transforms.Compose([
    transforms.CenterCrop(150),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
train_img = datasets.ImageFolder(train_dir, transform=train_transform)
test_img = datasets.ImageFolder(test_dir, transform=test_transform)

In [ ]:
print("train_img type   :",type(train_img)) # objek datasets
print("train_img length :",len(train_img)) # 940 data train
print("test_img length :",len(test_img)) # 120 data test
print("train_img classes:",train_img.classes) # nama kelas
print("train_img[0] type:",type(train_img[0])) # tuple
print("train_img[0][0] s:",train_img[0][0].size()) # tensor citra
print("train_img[0][1]  :",train_img[0][1]) # label

train_img type   : <class 'torchvision.datasets.folder.ImageFolder'>
train_img length : 940
test_img length : 120
train_img classes: ['JUICE', 'MILK', 'SODA', 'VINEGAR', 'WATER']
train_img[0] type: <class 'tuple'>
train_img[0][0] s: torch.Size([3, 150, 150])
train_img[0][1]  : 0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

trainloaders = torch.utils.data.DataLoader(train_img, batch_size=16, shuffle=True)
testloaders = torch.utils.data.DataLoader(test_img, batch_size=4, shuffle=True)

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Conv2d(3, 16, kernel_size=4, stride=1, padding=0)
    self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0)
    self.fc1 = nn.Linear(39200, 512)
    self.fc2 = nn.Linear(512, 5)
    self.do = nn.Dropout()
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.reshape(x.shape[0], -1)
    x = self.do(F.relu(self.fc1(x)))
    x = self.fc2(x)
    return x

In [ ]:
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
epoch = 20

model.train()
for i in range(epoch):
  total_loss = 0
  total_sample = 0
  total_correct = 0

  for image, label in trainloaders:

    out = model(image) # LANGKAH 1: forward propagation
    loss = criterion(out, label) # LANGKAH 2: hitung error
    optimizer.zero_grad() # LANGKAH 3: bersihkan nilai gradient sisa komputasi
    loss.backward() # LANGKAH 4: backpropagation
    optimizer.step() # LANGKAH 5: update bobot berdasar algoritma optimizer

    total_loss += loss.item() # untuk merata2 eror
    total_sample += len(label) # untuk merata2 eror dan akurasi
    total_correct += torch.sum(torch.max(out,1)[1]==label).item()*1.0 # untuk merata2 eror akurasi
    
  print("epoch", i, total_loss/total_sample, total_correct/total_sample) # menampilkan rata2 eror dan akurasi tiap epoch


epoch 0 0.03642831000875919 0.35425531914893615
epoch 1 0.020057505876459974 0.48723404255319147
epoch 2 0.019226529306553778 0.5159574468085106
epoch 3 0.017362424350799398 0.5606382978723404
epoch 4 0.015737651764078344 0.6148936170212767
epoch 5 0.014968176598244525 0.6457446808510638
epoch 6 0.013149521135269328 0.6936170212765957
epoch 7 0.012163667983197152 0.7202127659574468
epoch 8 0.010717109891962498 0.7510638297872341
epoch 9 0.009900979412362931 0.774468085106383
epoch 10 0.008516497719795147 0.7957446808510639
epoch 11 0.007513738566256584 0.8223404255319149
epoch 12 0.007487180638820567 0.8308510638297872
epoch 13 0.006850036344629653 0.85
epoch 14 0.005978196764245947 0.8670212765957447
epoch 15 0.005256506705537755 0.8840425531914894
epoch 16 0.005243601856079507 0.902127659574468
epoch 17 0.003850463682666738 0.9212765957446809
epoch 18 0.0036482975679509185 0.9212765957446809
epoch 19 0.0034922234396985237 0.9308510638297872


In [ ]:
model.eval()
total_loss = 0
total_sample = 0
total_correct = 0

for image, label in testloaders:
  out = model(image)
  loss = criterion(out, label)

  total_loss += loss.item()
  total_sample += len(label)
  total_correct += torch.sum(torch.max(out, 1)[1] == label).item()*1.0
print("test loss", total_loss/total_sample)
print("test accuracy", total_correct/total_sample)


test loss 0.03749907861153285
test accuracy 0.6583333333333333
